<a href="https://colab.research.google.com/github/simonbustamante/mit-algoritmos-de-regresion/blob/master/Caso_de_Estudio_2_5_El_efecto_de_la_posesi%C3%B3n_de_armas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Caso de Estudio 2.5 - El efecto de la posesión de armas en las tasas de homicidio


Configuración del *notebook*:

Sincronice su cuenta de Google. Para ello, siga el link que aparece en la salida de la siguiente celda una vez ejecutada. Copie el código que le aparece en pantalla e introdúzcalo en la salida de la celda. Una vez vea el mensaje: `Google Drive sincronizado con éxito!` puede continuar ejecutando el resto de celdas.

In [ ]:
from google.colab import auth
auth.authenticate_user()

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
data_drop = drive.CreateFile({'id':'1wRXji7OJE6mAs_obWZc0U_fHyW63JP82'})
data_drop.GetContentFile('gun_clean.csv')
data_drop = drive.CreateFile({'id':'1aIbjCQWQXHaCTWku1asywKeNq6dTTC9e'})
data_drop.GetContentFile('Cond-comp.R')
data_drop = drive.CreateFile({'id':'1IQqdJnEiSqXSfOgpRxozyxlthDYiW1bX'})
data_drop.GetContentFile('Functions.R')

print('Google Drive sincronizado con éxito!')

In [ ]:
!pip install rpy2==3.5.1
%load_ext rpy2.ipython

Instalar e importar librerías:

(ignorar resultados a no ser que no se muestre la frase: `Librerías instaladas con éxito!`)

(podría tardar unos cuantos minutos)

In [ ]:
%%R --noreturn
# Instalar librerías

install.packages("foreign")
install.packages("quantreg")
install.packages("mnormt")
install.packages("gbm")
install.packages("glmnet")
install.packages("MASS")
install.packages("rpart")
install.packages("sandwich")
install.packages("hdm")
install.packages("randomForest")
install.packages("nnet")
install.packages("neuralnet")
install.packages("caret")
install.packages("matrixStats")
install.packages("devtools")
install.packages("plyr")

cat('\n¡Librerías instaladas con éxito!')

Importar librerías:

In [ ]:
%%R
# Cargar librerías
library(foreign)
library(quantreg)
library(mnormt)
library(gbm)
library(glmnet)
library(MASS)
library(rpart)
library(sandwich)
library(hdm)
library(randomForest)
library(nnet)
library(neuralnet)
library(caret)
library(matrixStats)
library(devtools)
library(plyr)

cat('\n¡Librerías importadas con éxito!')

Importar funciones de archivos externos:

In [ ]:
%%R
source("Cond-comp.R")
source("Functions.R")

## Datos

In [ ]:
%%R
# Establecer generador de números aleatorios
set.seed(1)

# Cargar la base de datos
data <- read.csv("gun_clean.csv", sep = ",", header = TRUE)

# Tabla para guardar los resultados
table <- matrix(0,12,2)

Recuperar los nombres de las variables de la base de datos:

In [ ]:
%%R
varlist <- function (df=NULL,type=c("numeric","factor","character"), pattern="", exclude=NULL) {
  vars <- character(0)
  if (any(type %in% "numeric")) {
    vars <- c(vars,names(df)[sapply(df,is.numeric)])
  }
  if (any(type %in% "factor")) {
    vars <- c(vars,names(df)[sapply(df,is.factor)])
  }
  if (any(type %in% "character")) {
    vars <- c(vars,names(df)[sapply(df,is.character)])
  }
  vars[(!vars %in% exclude) & grepl(vars,pattern=pattern)]
}

Crear variables:

In [ ]:
%%R
# Variables binarias para los efectos temporales y de condados
fixed  <- grep("X_Jfips", names(data), value=TRUE, fixed=TRUE)
year   <- varlist(data, pattern="X_Tyear")

# Variables de control del censo
census     <- NULL
census_var <- c("^AGE", "^BN", "^BP", "^BZ", "^ED", "^EL","^HI", "^HS", "^INC", "^LF", "^LN", "^PI", "^PO", "^PP", "^PV", "^SPR", "^VS")

for(i in 1:length(census_var)){
  census  <- append(census, varlist(data, pattern=census_var[i]))
}

# Regresor objetivo (variable de tratamiento)
d     <- "logfssl"

# Variable de resultado
y     <- "logghomr"

# Otras variables de control
X1    <- c("logrobr", "logburg", "burg_missing", "robrate_missing")
X2    <- c("newblack", "newfhh", "newmove", "newdens", "newmal")

## Metodología

Extracción parcial de los efectos temporales y de condados:

In [ ]:
%%R
# Nuevo conjunto de datos para las variables parcialmente extraídas
rdata    <- as.data.frame(data$CountyCode)
colnames(rdata) <- "CountyCode"

# Variables a ser parcialmente extraídas
varlist <- c(y, d,X1, X2, census)

# Extracción parcial de los efectos temporales y de condado en las variables de varlist
for(i in 1:length(varlist)){
  form <- as.formula(paste(varlist[i], "~", paste(paste(year,collapse="+"),  paste(fixed,collapse="+"), sep="+")))
  rdata[, varlist[i]] <- lm(form, data)$residuals
}

Regresión lineal:

In [ ]:
%%R
form1 <- as.formula(paste(y, "~", d ))
form2 <- as.formula(paste(y, "~", paste(d, paste(X1,collapse="+"), paste(X2,collapse="+"), paste(census,collapse="+"),   sep="+")))

table[1,1:2] <- summary(lm(form1, rdata))$coef[2,1:2]
table[2,1:2] <- summary(lm(form2, rdata))$coef[2,1:2]

Métodos Dobles de Machine Learning (*Double Machine Learning Methods*, Chernozhukov et al., 2018):

(podría llevar unos minutos su ejecución)

In [ ]:
%%R
# Variable de resultado
y.name      <-  y;

# Regresor objetivo (también: variable de tratamiento)
d.name      <- d;

# Controles
x.name      <- paste(paste(X1,collapse="+"),  paste(X2,collapse="+"), paste(census,collapse="+"), sep="+") # use this for tree-based methods like forests and boosted trees

# Nombres de los métodos
method      <- c("RLasso","PostRLasso", "Forest", "Boosting", "Trees", "Lasso", "Ridge", "Elnet", "Nnet")

# Función que devuelve la estimación de coeficientes por Métodos Dobles de Machine Learning
# lista de argumentos:
#    1 : datos
#    2 : variable de resultado
#    3 : regresor objetivo
#    4 : variables de control para métodos basados en árboles
#    5 : variables de control para modelos lineales (especificación flexible) - 4 y 5 son iguales en este ejemplo
#    6 : nombres de los métodos
#    7 : número de separaciones de datos
#    8 : modelo parcialmente extraído

res <- DoubleML(rdata, y.name, d.name, x.name, x.name, method=method, 2 ,"plinear")

table[3,1:2]   <-res[,1]
table[4,1:2]   <-res[,2]
table[5,1:2]  <-res[,3]
table[6,1:2] <-res[,4]
table[7,1:2] <-res[,5]
table[8,1:2] <-res[,6]
table[9,1:2] <-res[,7]
table[10,1:2] <-res[,8]
table[11,1:2] <-res[,9]
table[12,1:2] <-res[,10]

## Resultados

In [ ]:
%%R
# Asignar nombres de columnas y filas
colnames(table) <- c("Estimación (beta)", "Error Estándar")
rownames(table) <- c("OLS (sin controles)", "OLS", "Lasso", "Post-Lasso","Random Forest",
                     "Boosting Trees","Árbol podado","Lasso con VC", "Ridge con VC",
                     "Elastic Net  con VC","Red Neuronal", "Mejor predictor")

# Mostrar resultados
print(table, digit=3)